#### LAB

# Advanced Querying Mongo

Importing libraries and setting up connection

In [4]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [5]:
client.list_database_names()

['IRONHACK', 'admin', 'config', 'local']

In [8]:
collection = client.get_database('IRONHACK').get_collection('companies')

In [9]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'IRONHACK'), 'companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [10]:
list(collection.find({'name':'Babelgum'},{'name':1, '_id':0}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [22]:
list(collection.find({'number_of_employees':{'$gt':5000}},{'name':1, 'number_of_employees':1, '_id':0}).limit(20).sort([('number_of_employees',-1)]))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [50]:

list(collection.find({"$or":[{"founded_year":2000}, {"founded_year":2005}]}, {"name":1, "founded_year":1, '_id':0}))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'YouTube', 'founded_year': 2005},
 {'name': 'Revision3', 'founded_year': 2005},
 {'name': 'iSkoot', 'founded_year': 2005},
 {'name': 'AllofMP3', 'founded_year': 2000},
 {'name': 'Steorn', 'founded_year': 2000},
 {'name': 'MeeVee', 'founded_year': 2000},
 {'name': 'ClipBlast!', 'founded_year': 2005},
 {'name': 'Zlio', 'founded_year': 2005},
 {'name': 'Dailymotion', 'founded_year': 2005},
 {'name': 'RockYou', 'founded_year': 2005},
 {'name': 'Instructables', 'founded_year': 2005},
 {'name': 'Netvibes', 'founded_year': 2005},
 {'name': 'Swivel', 'founded_year': 2005},
 {'name': 'Slide

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [51]:
list(collection.find({"$and":[{"ipo.valuation_amount": {"$gt": 10000000}}, {"founded_year": {"$lt": 2010}}]}, 
                    {"name":1, "ipo":1}).limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d94'),
  'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297de0'),
  'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e0c'),
  'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e7a

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [53]:
list(collection.find({"$and":[{"number_of_employees":{"$lt":1000}},{"founded_year":{"$lt":2005}}]}).sort("number_of_employess",1).limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd675297da7'),
  'name': 'AddThis',
  'permalink': 'addthis',
  'crunchbase_url': 'http://www.crunchbase.com/company/addthis',
  'homepage_url': 'http://www.addthis.com',
  'blog_url': 'http://addthis.com/blog',
  'blog_feed_url': 'http://addthis.com/blog/feed',
  'twitter_username': 'addthis',
  'category_code': 'advertising',
  'number_of_employees': 120,
  'founded_year': 2004,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'sharing, social-applications, social-media, analytics, audience-targeting, advertising-solutions, web-plugins, consumer-engagement-tools, big-data',
  'alias_list': 'Clearspring,Clearspring Technologies',
  'email_address': 'info@addthis.com',
  'phone_number': '703-677-3999',
  'description': 'Content Engagement',
  'created_at': 'Wed Jun 13 09:00:07 UTC 2007',
  'updated_at': 'Thu Dec 05 23:54:22 UTC 2013',
 

### 6. All the companies that don't include the `partners` field.

In [57]:
collection.find_one({"partners":1}) # No me sale ningún resultado, entiendo que no hay

### 7. All the companies that have a null type of value on the `category_code` field.

In [59]:
list(collection.find({"founded_year": 2006},{"category_code":1, "name":1}))

[{'_id': ObjectId('52cdef7c4bab8bd675297d91'),
  'name': 'Geni',
  'category_code': 'web'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d94'),
  'name': 'Twitter',
  'category_code': 'social'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d98'),
  'name': 'Slacker',
  'category_code': 'music'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9d'),
  'name': 'Joost',
  'category_code': 'games_video'},
 {'_id': ObjectId('52cdef7c4bab8bd675297da4'),
  'name': 'Powerset',
  'category_code': 'search'},
 {'_id': ObjectId('52cdef7c4bab8bd675297dab'),
  'name': 'Kyte',
  'category_code': 'games_video'},
 {'_id': ObjectId('52cdef7c4bab8bd675297dae'),
  'name': 'Thoof',
  'category_code': 'web'},
 {'_id': ObjectId('52cdef7c4bab8bd675297db9'),
  'name': 'Prosper',
  'category_code': 'finance'},
 {'_id': ObjectId('52cdef7c4bab8bd675297dbe'),
  'name': 'Stickam',
  'category_code': 'games_video'},
 {'_id': ObjectId('52cdef7c4bab8bd675297dbf'),
  'name': 'blogTV',
  'category_code': 'games_video'},
 {'_id': ObjectI

In [61]:
list(collection.find({"category_code": None}, {"name":1, "category_code":1, "_id":0}))

[{'name': 'Collective', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None},
 {'name': 'VidKing', 'category_code': None},
 {'name': 'Drigg', 'category_code': None},
 {'name': 'SpaceTime', 'category_code': None},
 {'name': 'Touch Clarity', 'category_code': None},
 {'name': 'MMDAYS', 'category_code': None},
 {'name': 'Inside Group', 'category_code': None},
 {'name': 'Repeater Store', 'category_code': None},
 {'name': 'Tapesh', 'category_code': None},
 {'name': 'iPersians', 'category_code': None},
 {'name': 'NewPersia', 'category_code': None},
 {'name': 'Pyra Labs', 'category_code': None},
 {'name': 'Feedmap', 'category_code': None},
 {'name': 'NuvoMedia', 'category_code': None},
 {'name': 'Intwine', 'category_code': None},
 {'name': 'The Weinstein Company', 'category_code': None},
 {'name': 'ExecuNet', 'category_code': None},
 {'name': 'A1Vacations', 'category_code': None},
 {'na

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [62]:
list(collection.find({"number_of_employees" : {"$gt":100, "$lt": 1000}}, 
                {"name":1, "number_of_employees": 1}).limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'number_of_employees': 600},
 {'_id': ObjectId('52cdef7c4bab8bd675297da7'),
  'name': 'AddThis',
  'number_of_employees': 120},
 {'_id': ObjectId('52cdef7c4bab8bd675297da8'),
  'name': 'OpenX',
  'number_of_employees': 305},
 {'_id': ObjectId('52cdef7c4bab8bd675297db5'),
  'name': 'LifeLock',
  'number_of_employees': 644},
 {'_id': ObjectId('52cdef7c4bab8bd675297dbb'),
  'name': 'Jajah',
  'number_of_employees': 110},
 {'_id': ObjectId('52cdef7c4bab8bd675297dc0'),
  'name': 'Livestream',
  'number_of_employees': 120},
 {'_id': ObjectId('52cdef7c4bab8bd675297dc1'),
  'name': 'Ustream',
  'number_of_employees': 250},
 {'_id': ObjectId('52cdef7c4bab8bd675297dda'),
  'name': 'iContact',
  'number_of_employees': 300},
 {'_id': ObjectId('52cdef7c4bab8bd675297de0'),
  'name': 'Yelp',
  'number_of_employees': 800},
 {'_id': ObjectId('52cdef7c4bab8bd675297dee'),
  'name': 'Dailymotion',
  'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [63]:
list(collection.find().sort("ipo",-1).limit(10))

[{'_id': ObjectId('52cdef7e4bab8bd67529a8b4'),
  'name': 'GREE',
  'permalink': 'gree',
  'crunchbase_url': 'http://www.crunchbase.com/company/gree',
  'homepage_url': 'http://www.gree-corp.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'gree_corp',
  'category_code': 'games_video',
  'number_of_employees': 700,
  'founded_year': 2004,
  'founded_month': 12,
  'founded_day': 7,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'mobile-web, japan, tokyo, social-network, mobile-social-network, mobile-games',
  'alias_list': None,
  'email_address': 'inquiry@gree-corp.com',
  'phone_number': '',
  'description': 'Internet media business,SNS,  free game',
  'created_at': 'Sat Dec 20 16:42:57 UTC 2008',
  'updated_at': 'Tue Jan 01 21:37:04 UTC 2013',
  'overview': '<p>GREE provides Japan&#8217;s leading mobile social network, and is at the forefront of mobile technology. GREE was ranked as Japan&#82

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [64]:
list(collection.find({},{"name":1, "number_of_employees":1}).sort("number_of_employees",-1).limit(10))

[{'_id': ObjectId('52cdef7d4bab8bd67529941a'),
  'name': 'Siemens',
  'number_of_employees': 405000},
 {'_id': ObjectId('52cdef7c4bab8bd67529856a'),
  'name': 'IBM',
  'number_of_employees': 388000},
 {'_id': ObjectId('52cdef7d4bab8bd675299d33'),
  'name': 'Toyota',
  'number_of_employees': 320000},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'),
  'name': 'PayPal',
  'number_of_employees': 300000},
 {'_id': ObjectId('52cdef7e4bab8bd67529b0fe'),
  'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'_id': ObjectId('52cdef7d4bab8bd675298aa4'),
  'name': 'Samsung Electronics',
  'number_of_employees': 221726},
 {'_id': ObjectId('52cdef7d4bab8bd675298b99'),
  'name': 'Accenture',
  'number_of_employees': 205000},
 {'_id': ObjectId('52cdef7e4bab8bd67529a657'),
  'name': 'Tata Consultancy Services',
  'number_of_employees': 200300},
 {'_id': ObjectId('52cdef7e4bab8bd67529aa51'),
  'name': 'Flextronics International',
  'number_of_employees': 200000},
 {'

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [65]:
list(collection.find({"founded_month": {"$gt":6}},{"name":1,"founded_month":1}).limit(1000))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'founded_month': 10},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'),
  'name': 'Zoho',
  'founded_month': 9},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'),
  'name': 'Digg',
  'founded_month': 10},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
  'name': 'Omnidrive',
  'founded_month': 11},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9b'),
  'name': 'eBay',
  'founded_month': 9},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9d'),
  'name': 'Joost',
  'founded_month': 10},
 {'_id': ObjectId('52cdef7c4bab8bd675297da1'),
  'name': 'Plaxo',
  'founded_month': 11},
 {'_id': ObjectId('52cdef7c4bab8bd675297da4'),
  'name': 'Powerset',
  'founded_month': 10},
 {'_id': ObjectId('52cdef7c4bab8bd675297dab'),
  'name': 'Kyte',
  'founded_month': 12},
 {'_id': ObjectId('52cdef7c4bab8bd675297dae'),
  'name': 'Thoof',
  'founded_month': 12},
 {'_id': ObjectId('52cdef7c4bab8bd675297daf'),
  'name': 'Jingle Networks',
  'founded_month': 9

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [66]:
list(collection.find({"$and":[{"founded_year": {"$lt": 2000}},{"ipo.valuation_amount":{"$gt": 10000}}]},
                {"name": 1, "founded_year": 1, "ipo.valuation_amount":1}).limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd675297e7a'),
  'name': 'Amazon',
  'founded_year': 1994,
  'ipo': {'valuation_amount': 100000000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297f15'),
  'name': 'Nielsen',
  'founded_year': 1922,
  'ipo': {'valuation_amount': 1600000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297f36'),
  'name': 'OpenTable',
  'founded_year': 1998,
  'ipo': {'valuation_amount': 1050000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675298112'),
  'name': 'Shutterfly',
  'founded_year': 1999,
  'ipo': {'valuation_amount': 350000000}},
 {'_id': ObjectId('52cdef7c4bab8bd67529823b'),
  'name': 'Salesforce',
  'founded_year': 1999,
  'ipo': {'valuation_amount': 110000000}},
 {'_id': ObjectId('52cdef7c4bab8bd6752984e3'),
  'name': 'QuinStreet',
  'founded_year': 1999,
  'ipo': {'valuation_amount': 140000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675298527'),
  'name': 'Rackspace',
  'founded_year': 1998,
  'ipo': {'valuation_amount': 5440000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [67]:
list(collection.find({"acquisition.acquired_year": {"$gt": 2010}},
                    {"name": 1, "acquisition": 1}).sort("acquisition.price_amount",-1).limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd675298876'),
  'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'_id': ObjectId('52cdef7f4bab8bd67529c228'),
  'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_comp

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [69]:
list(collection.find({}, {"name":1, "founded_year":1}).sort("founded_year", -1))

[{'_id': ObjectId('52cdef7c4bab8bd675297fec'),
  'name': 'Fixya',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd67529801f'),
  'name': 'Wamba',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd6752982d4'),
  'name': 'Advaliant',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd67529830a'),
  'name': 'Fluc',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7d4bab8bd675298ea7'),
  'name': 'iBazar',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7d4bab8bd675298fe1'),
  'name': 'Gimigo',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7d4bab8bd675299291'),
  'name': 'SEOGroup',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7e4bab8bd67529a3b8'),
  'name': 'Clowdy',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7e4bab8bd67529a6e7'),
  'name': 'WhosCall',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7e4bab8bd67529b254'),
  'name': 'Pikk',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7e4bab8bd67529b40e'),
  'name': 'Tongxue',
  'founded_

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [70]:
list(collection.find({"founded_day": {"$lt": 8}}, {"name": 1, "founded_day":1, "acquisition.price_amount":1}).sort(
    "acquisition.price_amount", -1).limit(10))

[{'_id': ObjectId('52cdef7d4bab8bd6752989a1'),
  'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'),
  'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297efe'),
  'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297f0c'),
  'name': 'Alibaba',
  'founded_day': 1,
  'acquisition': {'price_amount': 1000000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d90'),
  'name': 'Postini',
  'founded_day': 2,
  'acquisition': {'price_amount': 625000000}},
 {'_id': ObjectId('52cdef7c4bab8bd67529831c'),
  'name': 'Danger',
  'founded_day': 1,
  'acquisition': {'price_amount': 500000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675298651'),
  'name': 'Clearwell Systems',
  'founded_day': 6,
  'acquisition': {'price_amount': 410000000}},
 {'_id': ObjectId('52cdef7d4bab8bd6752989b8

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [71]:
list(collection.find({"category_code": "web", "number_of_employees": {"$gt": 4000}}, {
    "name":1, "category_code":1, "number_of_employees":1}).sort("number_of_employees",1))

[{'_id': ObjectId('52cdef7c4bab8bd67529822a'),
  'name': 'Expedia',
  'category_code': 'web',
  'number_of_employees': 4400},
 {'_id': ObjectId('52cdef7c4bab8bd675297e96'),
  'name': 'AOL',
  'category_code': 'web',
  'number_of_employees': 8000},
 {'_id': ObjectId('52cdef7c4bab8bd675297ea4'),
  'name': 'Webkinz',
  'category_code': 'web',
  'number_of_employees': 8657},
 {'_id': ObjectId('52cdef7c4bab8bd675297fcb'),
  'name': 'Rakuten',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'_id': ObjectId('52cdef7c4bab8bd67529834c'),
  'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'_id': ObjectId('52cdef7d4bab8bd675299d5d'),
  'name': 'Groupon',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'_id': ObjectId('52cdef7c4bab8bd675297da3'),
  'name': 'Yahoo!',
  'category_code': 'web',
  'number_of_employees': 13600},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9b'),
  'name': 'eBay',
  'category_code': 'web',
  'num

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [72]:
list(collection.find({"acquisition.price_amount": {"$gt": 10000}, "acquisition.price_currency_code": "USD"}, 
               {"name":1, "acquisition.price_amount":1, "acquisition.price_currency_code":1}))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'acquisition': {'price_amount': 30000000, 'price_currency_code': 'USD'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'),
  'name': 'Digg',
  'acquisition': {'price_amount': 500000, 'price_currency_code': 'USD'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d90'),
  'name': 'Postini',
  'acquisition': {'price_amount': 625000000, 'price_currency_code': 'USD'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d92'),
  'name': 'Flektor',
  'acquisition': {'price_amount': 20000000, 'price_currency_code': 'USD'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d95'),
  'name': 'StumbleUpon',
  'acquisition': {'price_amount': 29000000, 'price_currency_code': 'USD'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d99'),
  'name': 'Lala',
  'acquisition': {'price_amount': 80000000, 'price_currency_code': 'USD'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9a'),
  'name': 'Helio',
  'acquisition': {'price_amount': 39000000, 'price_currency_code': 'USD'

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [73]:
list(collection.find({"acquisition.acquired_month":{"$lt": 4}}, {"name":1, "acquisition":1}).limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd675297dab'),
  'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'_id': ObjectId('52cdef7c4bab8bd675297db4'),
  'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_descript

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [74]:
list(collection.find({"founded_year": {"$gt":2000, "$lt": 2010}, "acquisition.acquired_year": {"$gt": 2011}},
               {"name": 1, "founded_year":1, "acquisition":1}))


[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'founded_year': 2005,
  'acquisition': {'price_amount': 30000000,
   'price_currency_code': 'USD',
   'term_code': 'cash_and_stock',
   'source_url': 'http://allthingsd.com/20131216/viggle-tries-to-bulk-up-its-social-tv-business-by-buying-wetpaint/?mod=atdtweet',
   'source_description': ' Viggle Tries to Bulk Up Its Social TV Business by Buying Wetpaint',
   'acquired_year': 2013,
   'acquired_month': 12,
   'acquired_day': 16,
   'acquiring_company': {'name': 'Viggle', 'permalink': 'viggle'}}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'),
  'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'price_amount': 500000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2012/07/12/betaworks-acquires-digg/',
   'source_description': 'Betaworks Acquires Digg (TechCrunch)',
   'acquired_year': 2012,
   'acquired_month': 7,
   'acquired_day': 12,
   'acquiring_company': {

### 20. All the companies that have been 'deadpooled' after the third year.

In [75]:
result = list(collection.aggregate([
  {'$project':{'name':1,'_id':0,
      'Diff_year': { '$subtract': ['$deadpooled_year', '$founded_year']}}
  }, { "$match" : { 
        "Diff_year" : { "$gte" : 3 } 
    } }
]))

In [76]:
result

[{'name': 'Omnidrive', 'Diff_year': 3},
 {'name': 'Babelgum', 'Diff_year': 6},
 {'name': 'Thoof', 'Diff_year': 7},
 {'name': 'Wesabe', 'Diff_year': 5},
 {'name': 'Jangl SMS', 'Diff_year': 3},
 {'name': 'Stickam', 'Diff_year': 7},
 {'name': 'AllPeers', 'Diff_year': 4},
 {'name': 'EQO', 'Diff_year': 6},
 {'name': 'AllofMP3', 'Diff_year': 7},
 {'name': 'SellABand', 'Diff_year': 4},
 {'name': 'Zlio', 'Diff_year': 6},
 {'name': 'Jaiku', 'Diff_year': 6},
 {'name': 'Coghead', 'Diff_year': 6},
 {'name': 'Swivel', 'Diff_year': 5},
 {'name': 'Slide', 'Diff_year': 6},
 {'name': 'TripHub', 'Diff_year': 3},
 {'name': 'Sway', 'Diff_year': 8},
 {'name': 'coComment', 'Diff_year': 6},
 {'name': 'Hotelicopter', 'Diff_year': 6},
 {'name': 'Ujogo', 'Diff_year': 5},
 {'name': 'PeerMe', 'Diff_year': 5},
 {'name': 'Tangler', 'Diff_year': 6},
 {'name': 'TalkPlus', 'Diff_year': 4},
 {'name': 'PermissionTV', 'Diff_year': 7},
 {'name': 'Nirvanix', 'Diff_year': 6},
 {'name': 'Millsberry', 'Diff_year': 7},
 {'name